# Part 2: Integrating Tableau for Data Source Q&A

Welcome to the second part of our tutorial! In this section, we're going to take our Langchain agent to the next level by integrating it with **Tableau**. Now that you learned how to create an agent using the prebuilt `create_react_agent` in the [first exercise](./01_intro_to_langgraph.ipynb), let's provide this system with a Tableau tool so it can query a datasource and analyze the resulting data set.

The **Tableau Data Source Q&A tool** leverages a few powerful Tableau features behind the scenes to make this seamless integration possible:

1.  **VizQL Data Service API**: This is the communication channel our agent will use to request data from your published Tableau data sources. It also provides valuable metadata about the data.
2.  **Metadata API**: This API allows the agent to understand the semantics of your data, such as synonyms and descriptions for different fields, often managed through Tableau's Data Catalog.

With these capabilities you can equip the agent to surface insights in natural language from the data sources you have diligently built to run your analytics practice with [Tableau Prep](https://www.tableau.com/products/prep) (pictured below).

![Tableau Prep](./assets/prep_builder.png)

This integration offers a fantastic opportunity to amplify the impact of your data analytics work by making your Tableau data accessible and understandable through natural language queries.

In this exercise we will create a **Superstore Agent** that is able to query the [Superstore Datasource](https://help.tableau.com/current/guides/get-started-tutorial/en-us/get-started-tutorial-connect.htm). This is convenient because this data source is available on every [Tableau Cloud](https://www.tableau.com/products/cloud-bi) or [Tableau Server](https://www.tableau.com/products/server) site by default and it also ships with every [Tableau Desktop](https://www.tableau.com/products/desktop) app (so you can republish it if yours was deleted).

If you need a personal Tableau site for development purposes refer to [this guide](https://medium.com/@cristiansaavedra/how-to-get-your-tableau-online-sandbox-61d6a1f8b697).

Let's begin by importing the necessary Python packages for this part of the tutorial:

In [ ]:
# to access environment variables
import os
from dotenv import load_dotenv

# for displaying pretty results in the notebook
from IPython.display import display, Markdown

# Langgraph packages
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage # This will help with the markdown section

# langchain_tableau packages
from langchain_tableau.tools.simple_datasource_qa import initialize_simple_datasource_qa

### Environment Variables

In order to query published data sources on Tableau, the agent tool must authenticate to the site using a [Connected App](https://www.tableau.com/blog/unlock-power-personalized-analytics-user-attribute-functions). This authentication mechanism is the best practice for security and control when integrating code with Tableau as it provides the most control to developers.

Connected Apps [rely on secrets and IDs](https://help.tableau.com/current/online/en-us/connected_apps_direct.htm) to "sign" [JSON Web Tokens](https://jwt.io/) (JWTs) that should be kept private and not published to the internet on places such as Github. Other bad actors such as people and agents can find these credentials and use them for nefarios purposes. 

In order to secure these credentials we use the industry standard `.env` file which is ignored by `git` and therefore not published to places like Github when we share code.

The following code has the `load_dotenv()` function load the contents of the `.env` file so it is accessible via Python code and then used them to initialize the variables we need for the tool:

In [ ]:
# loads environment variables into Python script
load_dotenv()

# initialize variables with these secure values
tableau_server = os.getenv('TABLEAU_DOMAIN')
tableau_site = os.getenv('TABLEAU_SITE')
tableau_jwt_client_id = os.getenv('TABLEAU_JWT_CLIENT_ID')
tableau_jwt_secret_id = os.getenv('TABLEAU_JWT_SECRET_ID')
tableau_jwt_secret = os.getenv('TABLEAU_JWT_SECRET')
tableau_api_version = os.getenv('TABLEAU_API_VERSION')
tableau_user = os.getenv('TABLEAU_USER')
datasource_luid = os.getenv('DATASOURCE_LUID')
tooling_model = os.getenv('TOOLING_MODEL')

## Setting Up Our Tableau Data Source Q&A Tool

Now, let's bring our Tableau integration to life by initializing the necessary tool. We'll use the `initialize_simple_datasource_qa` function, providing it with all the information it needs to securely connect to and query your Tableau data source.

You'll notice that this function requires several key pieces of information:

* **Server URL**: The address of your Tableau Server or Tableau Cloud site.
* **Site ID**: The specific site on your Tableau Server or Tableau Cloud where your data source is published.
* **Connected App Credentials (JWT)**: The necessary authentication details (including the client ID and client secret) for the Connected App we discussed earlier.
* **Tableau User**: The username of the Tableau user the agent will act as when accessing the data source.
* **Data Source LUID**: The unique identifier (LUID) of the specific Tableau data source you want the agent to query.
* **Language Model (`model`)**: The LLM we initialized in the previous notebook (like `gpt-4o-mini`), which will be used to understand the user's questions and translate them into data queries.

The `datasource_luid` in this case should match the luid provided by your Tableau site for the *Superstore* data source. We suggest using the `GraphiQL` interface provided by Tableau to [query the Metadata API](https://www.tableau.com/developer/learning/metadata-api#tab-325797-1) to get this information.

The image below shows this *Superstore* data source in use in [Tableau Desktop](https://help.tableau.com/current/guides/get-started-tutorial/en-us/get-started-tutorial-drag.htm):

![.gif of Superstore being used in Tableau desktop](https://help.tableau.com/current/guides/get-started-tutorial/en-us/Img/Drag24.gif)

### The VDS API

Your agents will be able to interact with this data source in similar ways via Tableau's [VDS API](https://www.tableau.com/blog/vizql-data-service-beyond-visualizations). The VDS API able to fetch the same data for your code or agent, handling aggregations, applying filters (such as dates or topN) and resolve Tableau calculations. 

This is a sample query (in JSON) sent to VDS requesting the `Category`, `Ship Mode` and `Sales` while writing a new calculation called `AOV` filtered by a "Set filter" applied to `Ship Mode` so it only shows results for "First Class" and "Standard Class": 

```json
{
    "fields": [
        {
            "fieldCaption": "Category"
        },
        {
            "fieldCaption": "Ship Mode"
        },
        {
            "fieldCaption": "Sales",
            "function": "SUM"
        },
        {
                "fieldCaption": "AOV",
                "calculation": "SUM([Profit])/COUNTD([Order ID])"
            }
    ],
    "filters": [
        {
            "field": {
                "fieldCaption": "Ship Mode"
            },
            "filterType": "SET",
            "values": [ "First Class", "Standard Class" ],
            "exclude": "false"
        }
    ]
}
```

This makes the VDS API very powerful and fully accessible to AI Agents. It allows can query data in the exact shape you want and write new columns of data via calculations to surface new insights with the same data!


![postman logo](./assets/postman_logo.png)


We made those new VDS APIs easy to explore with a new addition to our [Tableau API Postman](https://www.postman.com/salesforce-developers/salesforce-developers/folder/ydjw53q/vizql-data-service) collection!


### Initializing the Tool

All of this information is crucial for the tool to:

1.  **Authenticate** to your Tableau site as a specific user.
2.  **Locate** the correct data source using its unique LUID and obtain metadata that describes it.
3.  **Leverage the LLM** to understand natural language questions and generate appropriate data queries against the Tableau data source.

Once the tool is successfully initialized, we'll add it to a Python list (`[]`). This is how we provide our agent with the set of tools it can use. In this case, we'll start with just our Tableau data source Q&A tool, but in more complex scenarios, this list could contain multiple tools with different capabilities.



In [91]:
# Large Language Model for writing VDS queries from natural language within the tool
tooling_model = 'gpt-4o-mini'

# Initalize the tool for querying Tableau Datasources through VDS
analyze_datasource = initialize_simple_datasource_qa(
    domain=tableau_server,
    site=tableau_site,
    jwt_client_id=tableau_jwt_client_id,
    jwt_secret_id=tableau_jwt_secret_id,
    jwt_secret=tableau_jwt_secret,
    tableau_api_version=tableau_api_version,
    tableau_user=tableau_user,
    datasource_luid=datasource_luid,
    tooling_llm_model=tooling_model
)

# add the tool to a List to give to the agent
tools = [ analyze_datasource ]

## The Superstore Agent

It is finally time to build our Superstore Agent with access to the [Superstore Data Source](https://help.tableau.com/current/guides/get-started-tutorial/en-us/get-started-tutorial-connect.htm).

After initializing the tool we can start an agent using the prebuilt `create_react_agent` function provided by Langchain. This function has three basic requirements: a model, a list of tools and a prompt to customize its behavior.

### Giving Our Agent a Tableau Superstore Persona!

Before we equip our agent with the ability to interact with Tableau, let's give it a distinct personality! We want it to feel like a knowledgeable Tableau expert. We can achieve this by defining an **agent system prompt**.

Think of the system prompt as the foundational instructions and guidelines that shape our agent's behavior, its tone, and even some of its core knowledge. It's like giving our agent a specific role to play.

Below, you'll find the system prompt we've carefully crafted for our Tableau Superstore expert. We encourage you to take a look and even tweak it if you're feeling adventurous! Just be aware that any changes you make to this prompt can influence how the agent behaves and interprets your queries.

Don't worry too much about breaking things! If your modifications inadvertently affect the data retrieval capabilities we'll be exploring later, we'll make sure you have access to the original, tested prompt to get back on track. So, feel free to experiment and see how different instructions can shape your Tableau-savvy agent!

In [ ]:
# Agent Identity Definition
identity = """
You are **Agent Superstore**, the veteran AI analyst who has spent years exploring the aisles of the legendary Superstore dataset.
A dataset many Tableau users know and love! 
You live and breathe Superstore data: sales, profits, regions, categories, customer segments, shipping modes, you name it.

Your special mission **today at Tableau Conference 2025** is to help attendees experience the power of Tableau for Langchain Agents. 
You'll be their guide, using this new tool to query the Superstore dataset directly and uncover insights in real-time.

**When you first introduce yourself:**
1.  Greet the attendees and welcome them to the Tableau Conference 2025 hands-on session.
2.  Introduce yourself as Agent Superstore, the AI expert on the classic Superstore dataset.
3.  Briefly explain your purpose: to demonstrate Tableau analytics via agents
"""

# Main System Prompt
system_prompt = f"""**Agent Identity:**
{identity}

**Core Instructions:**

You are an AI Analyst specifically designed to generate data-driven insights from datasets using the tools provided. 
Your goal is to provide answers, guidance, and analysis based on the data accessed via your tools. 
Remember your audience: Tableau users at a conference session, likely familiar with Superstore aka the best dataset ever created.

**Tool Usage Strategy:**

You have access to the following tool:

1.  **`tableau_query_tool` (Data Source Query):** This is your primary tool for interacting with data.
    * **Prioritize this tool** for nearly all user requests asking for specific data points, aggregations, comparisons, trends, or filtered information from datasets.
    * Use it to find specific values (e.g., sales for 'Technology' in 'West' region), calculate aggregates (e.g., `SUM(Sales)`, `AVG(Profit Ratio)`), filter data (e.g., orders in 2023), group data (e.g., sales `BY Category`), and find rankings (e.g., top 5 products by quantity).
    * Be precise in formulating the queries based on the user's request.

**Response Guidelines:**

* **Grounding:** Base ALL your answers strictly on the information retrieved from your available tools.
* **Clarity:** Always answer the user's core question directly first.
* **Source Attribution:** Clearly state that the information comes from the **dataset** accessed via the Tableau tool (e.g., "According to the data...", "Querying the datasource reveals...").
* **Structure:** Present findings clearly. Use lists or summaries for complex results like rankings or multiple data points. Think like a mini-report derived *directly* from the data query.
* **Tone:** Maintain a helpful, and knowledgeable, befitting your Tableau Superstore expert persona.

**Crucial Restrictions:**
* **DO NOT HALLUCINATE:** Never invent data, categories, regions, or metrics that are not present in the output of your tools. If the tool doesn't provide the answer, state that the information isn't available in the queried data.
"""


The LLM model in this case is the "brains" of the agent. This model chooses what tool to use according to the task is has been assigned to execute. In this capacity the LLM is relied upon to reason, plan and execute actions to accomplish a job.


We the initialize the Superstore Agent with the three basic components we now have:

- Model (*Large Language Model or the brains of the operation*)
- Tools (*for now this only contains the tool to query Tableau data sources*)
- Prompt (*instructions to customize the agent to help run the Superstore business*)

In [ ]:
# initialize a Large Language Model to be the "brains" of the Agent
model = ChatOpenAI(model='gpt-4o-mini', temperature=0)

superstore_agent = create_react_agent(model=model, tools=tools, prompt=system_prompt)

### Peeking Behind the Scenes: Observing the Agent's Reasoning

Before we unleash our Tableau-savvy agent, let's set up a way to observe its thought process. Understanding how the agent arrives at its answers can be incredibly insightful for debugging and gaining a deeper understanding of its behavior.

The following code defines a function that will allow us to see the agent's "chain of thought." This will show us the intermediate steps the agent takes, including its reasoning, the tools it considers using, and the actions it ultimately decides to take.

By running our queries through this function, you'll get a fascinating glimpse into how the agent interprets your questions and formulates its responses using the Tableau data source. Let's define this observational function:

In [ ]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

## Let's Meet Our Superstore Expert!

Alright, the moment has arrived to interact with our Tableau Superstore expert! But before we dive into querying our data, let's first ask the agent to introduce itself. This will give us a chance to see our carefully crafted system prompt in action and get a sense of the agent's defined personality.

Go ahead and run the following code snippet. We're asking a simple question that should prompt the agent to draw upon the instructions we provided in the system prompt. Let's see how our Tableau-focused agent responds! If you want to know more, feel free to modify the prompt.

In [ ]:
your_prompt = 'who are you?'

# Run the agent
messages = {"messages": [("user", your_prompt)]}
print_stream(superstore_agent.stream(messages, stream_mode="values"))

### Your Turn to Ask!

Now it's your chance to directly interact with our Tableau-integrated agent! In the code cell below, you'll find an example question: "Give me the top three states by profit."

Feel free to **replace this example with any natural language question you have about the Superstore dataset.** Think about the kinds of insights you'd typically look for in Tableau, and ask our agent! Here are a few more ideas to get you started:

* "What are the total sales for each region?"
* "Show me the sub-categories with the highest sales in the West."
* "What is the average discount applied to furniture orders?"
* "Which customer segment is the most profitable?"
* "Tell me the sales trend over in 2023."

Go ahead, type in your question and run the code cell. Let's see the power of combining natural language with Tableau analytics!

In [ ]:
your_prompt = '[ENTER YOUR PROMPT HERE]'

# Run the agent
messages = {"messages": [("user", your_prompt)]}
print_stream(superstore_agent.stream(messages, stream_mode="values"))

### Isn't that cool?

The same data powering your dashboards is now accessible to AI Agents to help them run workflows more reliably and save you from having to answer basic questions all the time in your company chat app!

Finally you get some time back to concentrate and plan your next data project for Superstore while agents help you with the low-hanging fruit.

![Superstore dashboard](https://help.tableau.com/current/guides/get-started-tutorial/en-us/Img/build6.png)

### Enhancing Readability: Formatting Agent Output with Markdown

You're absolutely right! While our agent is providing valuable insights, the raw text output can be a bit challenging to read and interpret, especially when dealing with data and analytics. Clear and understandable presentation is key!

To address this, we're going to introduce a simple yet powerful technique: **formatting the agent's output using Markdown**. Markdown is a lightweight markup language that allows us to easily add structure and style to plain text.

By implementing a function that helps the agent identify its output and format it using Markdown syntax, we can significantly improve the visual appeal and readability of the information it provides. This will allow the agent to present data in a more organized way, potentially using headings, bullet points, tables, and other Markdown elements, making the insights much easier on the eyes and quicker to grasp. Let's get our agent to present its findings in a more visually appealing manner!

In [ ]:
def run_agent_and_display_markdown(agent, user_prompt):
    """Invokes the agent and displays the final answer using Markdown."""
    print(f"Running agent with prompt: '{user_prompt}'")
    messages = {"messages": [HumanMessage(content=user_prompt)]} # Use HumanMessage directly

    try:
        # Invoke the agent to get the final result
        # The result dictionary structure might vary slightly depending on the LangGraph version
        # but typically contains 'messages' with the conversation history.
        result = agent.invoke(messages)

        # Extract the final message (usually the last AIMessage)
        if result and 'messages' in result and result['messages']:
            final_message = result['messages'][-1]
            # Check if it's an AI message and has content
            if hasattr(final_message, 'content'):
                final_answer = final_message.content
                print("\n--- Agent Final Answer ---")
                display(Markdown(final_answer))
                print("--------------------------\n")
            else:
                print("Could not extract content from the final message.")
                print("Final message:", final_message)
        else:
            print("Agent did not return the expected result structure.")
            print("Result:", result)

    except Exception as e:
        print(f"An error occurred while running the agent: {e}")
        # Potentially display the error or log it
        display(Markdown(f"**Error:**\n```\n{e}\n```"))


### Time to See the Markdown in action!

Our output formatting function is all set and ready to go! Now, let's put it to the test and see how it enhances the readability of our agent's responses.

In the code cell below, you'll find a placeholder: `"[ENTER YOUR PROMPT HERE]"`. Just like before, it's your turn to enter a natural language question about the Superstore dataset. Feel free to reuse a prompt you tried earlier or come up with a brand new one! Here are a few examples to inspire you:

* "Show me the total profit by region"
* "List the top 5 customers by sales"
* "Compare the sales of furniture and technology categories."

Go ahead and replace the placeholder with your chosen prompt and run the code cell. Let's see the difference that Markdown formatting makes in presenting the insights from our Tableau data!

In [ ]:
your_prompt = '[ENTER YOUR PROMPT HERE]'

# Run the agent with markdown
run_agent_and_display_markdown(superstore_agent, your_prompt)

### Unleashing the Power of LLM Reasoning with Fetched Data!

Now that our agent can successfully retrieve data from Tableau, let's explore the incredible reasoning capabilities of the underlying Large Language Model (LLM). As long as our requests align with the guidelines set in the system prompt, we can instruct the LLM to perform a wide variety of tasks with the data it fetches.

Let's try pushing the boundaries with some creative prompts:

**1. Generating a Customer Email Based on Data:**

For this task, we'll ask the agent to first fetch specific data and then use that information to compose a personalized email to a key customer. Imagine we want to inform a high-value customer about their recent purchase trends.

**2. Translating Data Insights:**

Next, we'll leverage the LLM's translation capabilities. We'll ask the agent to fetch some data and then translate the key insights into another language, such as Swedish or French, for one of your international colleagues. This showcases how the agent can bridge language barriers.

**3. Creative Data Output:**

Finally, let's get creative! We'll craft a prompt that asks the agent to fetch data and then present it in a more imaginative format. This could be a song, a poem, a rap, or any other creative expression the LLM can generate.

Go ahead and try crafting prompts that combine data fetching with these reasoning tasks in the code cell below. Remember to replace the placeholder with your creative instructions!

In [ ]:
your_prompt = '[ENTER YOUR PROMPT HERE]'

# Run the agent with markdown
run_agent_and_display_markdown(superstore_agent, your_prompt)

## Congratulations! You finished Part 2! 🎉

We now have a fully custom Superstore Agent ready to rock & roll! 

Fantastic work. You've successfully navigated Part Two of our hands-on training and witnessed the power of integrating Tableau with a Langchain agent. You've seen how the agent can query your data sources and provide valuable insights in natural language, even formatting its responses for better readability.

But hold on to your hats – the adventure isn't over yet! We're about to take things up a notch.

In our next step, we'll explore how to bring **even more tools into the mix**. Imagine an agent that can not only retrieve and analyze data from Tableau but also leverage other tools to perform additional actions based on those insights.

Get ready to see how we can make our agent even more versatile and useful by combining its data retrieval capabilities with a broader range of functionalities! Let's continue our journey and unlock even greater potential!

![blue dark line chart](./assets/vizart/lines-blue-dark.png)